
---
# Load needed libraries and functions

In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
%pprint

notebook_path = get_notebook_path()
print(notebook_path)

s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
hunting_df = s.load_object('hunting_df')
basic_quals_dict = s.load_object('basic_quals_dict')
dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\Miscellaneous\ipynb\Job Hunting.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'CountVectorizer', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', 'TfidfTransformer', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'basic_quals_dict', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'exit', 'get_classifier', 'get_data_structs_df', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_module_version', 'get_notebook_path', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'hunting_df', 'ipykernel', 'json', 'jupyter_config_dir', 'notebook_path', 'notebookapp', 'np', 'os', 'pd', 'pickle', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_starting_with', 'quit', 're', 'remove_empty_folders', 's', 'similar', 'subproc


---
# Needed extra functions

In [3]:

# Email prep
subject_str = '{}% fit: Internal Candidate, Dave Babbitt, for {}'
concerns_str = 'One important question I have is if the work can be supported remotely or if this position is available for remote delivery '
concerns_str += '(or something equivalent).'
concerns_str += " I don't want to move my family out of New England."
text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
emails_dir = os.path.join(s.saves_folder, 'emails')
os.makedirs(name=emails_dir, exist_ok=True)
name_regex = re.compile(r'^([^(]+) \(\d+\)"')
def clean_email(email_str):
    match_obj = name_regex.search(email_str)
    if match_obj:
        email_str = match_obj.group(1)
    names_list = re.split(r'\s+', email_str, 0)
    if len(names_list) >= 2:
        first_name = names_list[0]
        last_name = names_list[1]
        email_str = '{}, {} [USA] <{}_{}@bah.com>'.format(last_name, first_name, last_name.lower(), first_name.lower())
    
    return email_str

In [4]:

def get_percent_fit(row_series):
    percent_fit = row_series['percent_fit']
    if str(percent_fit) == 'nan':
        percent_fit = 0
    percent_fit = int(percent_fit*100)
    
    return percent_fit

In [5]:

def print_loc_computation(row_index, quals_list, basic_quals_dict):
    print()
    numerator_str_list = []
    for qual_str in quals_list:
        if qual_str in basic_quals_dict:
            numerator_str_list.append(str(basic_quals_dict[qual_str]))
        else:
            numerator_str_list.append('000')
    numerator_str = '+'.join(numerator_str_list)
    print("hunting_df.loc[{}, 'percent_fit'] = ({})/{}".format(row_index, numerator_str, len(quals_list)))

In [6]:

def get_predictions_and_counts(prediction_list, quals_list):
    qual_count = 0
    prediction_str = ''
    for pred_array, qual_str in zip(prediction_list, quals_list):
        prediction = pred_array[1]
        prediction_str += '\n{} {}'.format(prediction, qual_str)
        if prediction > 0.5:
            qual_count += 1
    
    return prediction_str, qual_count

In [7]:

def get_quals_str(prediction_list, quals_list, basic_quals_dict):
    qual_count = 0
    quals_str = ''
    for pred_array, (i, qual_str) in zip(prediction_list, enumerate(quals_list)):
        if qual_str in basic_quals_dict:
            formatted_str = '\nquals_list[{}] = "{}" ({})'
        else:
            formatted_str = '\n*quals_list[{}] = "{}" ({})'
        prediction = pred_array[1]
        quals_str += formatted_str.format(i, qual_str, prediction)
        if prediction > 0.5:
            qual_count += 1
    
    return quals_str, qual_count

In [8]:

def print_fit_job(row_index, row_series, basic_quals_dict):
    job_fitness = 0.0
    job_description = row_series['Job Description']
    quals_list = get_quals_list(job_description)
    if len(quals_list):
        prediction_list = list(predict_percent_fit(quals_list))
        #prediction_str, qual_count = get_predictions_and_counts(prediction_list, quals_list)
        quals_str, qual_count = get_quals_str(prediction_list, quals_list, basic_quals_dict)
        job_fitness = qual_count/len(prediction_list)
        if job_fitness > 0.8:
            print('Basic Qualifications:{}'.format(quals_str))
            #print(prediction_str)
            print(job_fitness)
            print_loc_computation(row_index, quals_list, basic_quals_dict)
    
    return quals_list, job_fitness

In [9]:

def qual_sum(qual_str):
    results = '"{}"'.format(qual_str)
    if qual_str in basic_quals_dict:
        results = basic_quals_dict[qual_str]
    else:
        results = predict_percent_fit([qual_str])[0][1]
        if results > 0.5:
            results = 1.0
        else:
            results = 0.0
    
    return str(results)

In [10]:

def print_job_description(req_id):
    match_series = (hunting_df['Job Requisition ID'] == req_id)
    job_description = hunting_df[match_series]['Job Description'].tolist()[0]
    print(get_quals_list(job_description))
    print(job_description)

In [11]:

scanner_regex = re.compile(r'\b[1-9a-zA-Z][0-9a-zA-Z]*( *[#\+]{1,2}|\b)')
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]

In [259]:

a_list = ['Additional Qualifications?', 'Nice If You Have', 'Nice if you have', 'Nice if You Have',
          'Additional Preferred Qualifications', 'Nice if you Have', 'Additional qualifications', 'Nice to Have']
a_str = '({}):?'.format('|'.join(a_list))
def get_quals_list(job_description):
    job_description = re.sub('Â', '', job_description)
    basic_quals = ''
    quals_list = []
    items_list = re.split('(Key Role|The Challenge):', job_description, 0)
    if len(items_list) > 1:
        job_description = items_list[-1].strip()
    items_list = re.split('[\r\n]+(Basic Qualifications?|You Have|You have):?', job_description, 0)
    if len(items_list) > 1:
        job_description = items_list[-1].strip()
    items_list = re.split(a_str, job_description, 0)
    if len(items_list) > 1:
        basic_quals = items_list[0].strip()
    else:
        items_list = re.split('(Clearance|Build Your Career):', job_description, 0)
        basic_quals = items_list[0].strip()
    if basic_quals != '':
        quals_list = [re.sub(r'â¯', ' ', q) for q in re.split('[\r\n]+', basic_quals, 0)]
        quals_list = [re.sub(r'^[?â-]+', '', x).strip() for x in quals_list]
        quals_list = [re.sub(r'[â-]+$', '', x).strip() for x in quals_list]
        quals_list = [re.sub(r'â', '-', x).strip() for x in quals_list]
        quals_list = [re.sub(r'â', '`', x).strip() for x in quals_list]
        quals_list = [re.sub(r'â', '`', x).strip() for x in quals_list]
        quals_list = [re.sub(r'â', '`', x).strip() for x in quals_list]
        quals_list = [re.sub(r'â', '', x).strip() for x in quals_list]
        quals_list = [re.sub(u'\\xa0', u' ', x).strip() for x in quals_list]
        quals_list = [re.sub(r'\s+$', '', x) for x in quals_list]
        quals_list = [x for x in quals_list if x != '']
    
    return quals_list

In [13]:

def print_emails(match_series):
    for file_name in os.listdir(emails_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(emails_dir, file_name)
            os.remove(file_path)
    for row_index, row_series in hunting_df[match_series].sort_values('percent_fit', ascending=False).iterrows():
        req_str = row_series['Job Requisition']
        percent_fit = get_percent_fit(row_series)
        sents_list = ['I’m submitting my resume for {}.'.format(req_str),
                      'I’ve reviewed the basic qualifications and believe I’m a good fit for this project.',
                      'Below is a breakdown of the requirements and the amount of experience I have with each.',
                      'I’m available at your convenience to discuss my qualifications and look forward to hearing from you.']
        blurb_str = ' '.join(sents_list)
        hm_str = row_series['Hiring Manager']
        pr_str = row_series['Primary Recruiter']
        job_description = row_series['Job Description']
        quals_list = get_quals_list(job_description)
        quals_str = '\n•\t' + '\n•\t'.join(quals_list)
        file_path = os.path.join(emails_dir, '{}_email.txt'.format(row_series['Job Requisition ID'].strip()))
        if not os.path.isfile(file_path):
            with open(file_path, 'w', encoding=s.encoding_type) as io_wrapper:
                print('', file=io_wrapper)
                print('To: {}; {}'.format(clean_email(hm_str), clean_email(pr_str)), file=io_wrapper)
                print('CC: Safi, Claudia [USA] <safi_claudia@bah.com>; Borrelli, Bill [USA] <Borrelli_Bill@bah.com>', file=io_wrapper)
                print(subject_str.format(percent_fit, req_str), file=io_wrapper)
                print('', file=io_wrapper)
                print('Dear {},'.format(hm_str.split(' ')[0]), file=io_wrapper)
                print('', file=io_wrapper)
                print('{}'.format(blurb_str), file=io_wrapper)
                print('', file=io_wrapper)
                print('Basic Qualifications:{}'.format(quals_str), file=io_wrapper)
                print('', file=io_wrapper)
                print(concerns_str, file=io_wrapper)
                print('', file=io_wrapper)
                print('Attached: Dave_Babbitt_Resume_for_{}.pdf'.format('_'.join(re.split(r'[ \\\/:\*\?"><\|]+', req_str, 0))),
                      file=io_wrapper)
            !"{text_editor_path}" "{os.path.abspath(file_path)}"
    !start %windir%\explorer.exe "{os.path.abspath(emails_dir)}"

In [14]:

# Add new ORR to the hunting dataframe
text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
jd_cn = 'Job Description'
reqid_cn = 'Job Requisition ID'
dupe_columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                     'Job Posting Title', 'Recruiting Start Date', 'Account Group', reqid_cn, 'Job Type',
                     'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                     'Resource Manager', 'Primary Location', jd_cn, 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                     'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                     'Required Clearance', 'Primary Recruiter']
columns_list = ['Job Posting', 'Job Requisition', reqid_cn, 'Job Requisition Status', 'Furthest Stage',
                'Supervisory Organization', 'Group', 'Account Group', 'IMT', 'Cluster', 'FSO', 'Primary Recruiter',
                'Resource Manager', 'Hiring Manager', 'Job Posting Title', 'Job Profile', 'Job Requisition Type',
                'Management Level', 'Primary Location', 'Primary Location State/Province', 'Primary Location Country',
                'Required Clearance', 'Clearance Agency', 'Time Type', 'Recruiting Start Date', 'Job Type', 'Job Family',
                'Business Title', 'Job Family Group', jd_cn]
hunting_dir = r'D:\Documents\Administrivia\Job Hunting\csv'
def add_new_orr(file_name, hunting_df):
    file_path = os.path.join(hunting_dir, file_name)
    if os.path.isfile(file_path):
        !"{text_editor_path}" "{os.path.abspath(file_path)}"
        df = pd.read_csv(file_path, header=0, skiprows=0, encoding='iso8859-1')
        df.columns = columns_list
        req_id_list = hunting_df[reqid_cn].unique().tolist()
        match_series = (df[reqid_cn].isin(req_id_list))
        hunting_df = pd.concat([hunting_df, df[~match_series]]).fillna({'is_opportunity_application_emailed': False})
        hunting_df[jd_cn] = hunting_df[jd_cn].map(lambda x: re.sub(u'\\xa0', u' ', x))
        hunting_df[jd_cn] = hunting_df[jd_cn].map(lambda x: re.sub(r'', "'", x))
        hunting_df[jd_cn] = hunting_df[jd_cn].map(lambda x: re.sub(r'', '—', x))
        hunting_df = hunting_df.drop_duplicates(subset=dupe_columns_list, ignore_index=True)
        hunting_df.reset_index(drop=True, inplace=True)
        
        return hunting_df

In [15]:

basic_quals_dict = s.load_object('basic_quals_dict')
def get_basic_quals(row_index):
    match_series = (hunting_df.index == row_index)
    for row_index, row_series in hunting_df[match_series].iterrows():
        percent_fit = get_percent_fit(row_series)
        job_description = row_series['Job Description']
        quals_list = get_quals_list(job_description)
        quals_str = get_quals_str(quals_list, basic_quals_dict)
        if (quals_str != '') and (percent_fit == 0):
            print('Basic Qualifications:{}'.format(quals_str))
            print_loc_computation(row_index, quals_list, basic_quals_dict)
    
    return quals_list


---
# Create the emails

In [322]:

def f(x):
    
    return 'python' in str(x).lower()

match_series = hunting_df['Job Requisition Type'].isin(['Sold and Funded', 'Sold & Unfunded']) & hunting_df['Job Description'].map(f)
match_series = match_series & (hunting_df.percent_fit >= 0.95) & ~hunting_df['is_opportunity_application_emailed']
match_series = match_series & ~(hunting_df.is_remote_delivery == False)
match_series = match_series & ~hunting_df['Required Clearance'].isin(['TS/SCI', 'TS/SCI w/CIP'])
print(hunting_df[match_series].shape)
print(hunting_df[match_series].groupby('Required Clearance').count().T.max().sort_values(ascending=False))
hunting_df[match_series].head(5).T

(3, 35)
Required Clearance
Secret                                2
Eligibility Determination Timeline    1
dtype: int64


,3994,4006,4008
Hiring Manager,Gregory Dupier (514538),Robin Rudy (509009),Sivakumar Arunachalam | Siva Arunachalam (608203)
Management Level,Senior Consultant,Senior Consultant,Senior Consultant
IMT,DIGITAL IMT,JHT IMT,JCC IMT
Job Requisition,R0080116 Data Analyst (Open),R0080071 Data Scientist (Open),"R0079006 Data Scientist, Mid (Open)"
Job Requisition Type,Sold and Funded,Sold and Funded,Sold and Funded
Cluster,Aberdeen Cluster,Wash Metro Cluster,Wash Metro Cluster
Time Type,Full time,Full time,Full time
Job Posting Title,Data Analyst,Data Scientist,"Data Scientist, Mid"
Safi Recommendation,NaN,NaN,NaN
Recruiting Start Date,2020-03-04,2020-03-04,2020-02-19


In [323]:

print_emails(match_series)

In [326]:

# Manually note the email has been sent
req_id = 'R0080116'.strip()
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle



---

In [319]:

# Rebuild the datframe from the dictionary
rows_list = [{'qualification_str': qualification_str, 'is_fit': is_fit} for qualification_str, is_fit in basic_quals_dict.items()]
basic_quals_df = pd.DataFrame(rows_list)
s.store_objects(basic_quals_df=basic_quals_df)

# Re-transform the bag-of-words and tf-idf from the new manual scores
sents_list = basic_quals_df.qualification_str.tolist()

# Bag-of-words
cv = CountVectorizer(lowercase=True, tokenizer=regex_tokenizer, token_pattern=r'\b[1-9a-zA-Z][0-9a-zA-Z]*[#\+]{0,2}', ngram_range=(1, 3))
bow_matrix = cv.fit_transform(sents_list)
s.store_objects(bq_cv_vocab=cv.vocabulary_)

# Tf-idf, must get from BOW first
tt = TfidfTransformer()
tfidf_matrix = tt.fit_transform(bow_matrix)
s.store_objects(bq_tt=tt)

# Re-train the classifier
X = tfidf_matrix.toarray()
y = basic_quals_df.is_fit.to_numpy()
fit_estimators_dict = s.load_object('fit_estimators_dict')
#basic_quals_clf = RandomForestClassifier(n_estimators=997)
#basic_quals_clf = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0, n_estimators=50, random_state=None)
basic_quals_clf = fit_estimators_dict['LogisticRegression']
basic_quals_clf.fit(X, y)
s.store_objects(basic_quals_clf=basic_quals_clf)

# Re-calibrate the inference engine
bq_cv_vocab = s.load_object('bq_cv_vocab')
bq_cv = CountVectorizer(vocabulary=bq_cv_vocab)
bq_cv._validate_vocabulary()
bq_tt = s.load_object('bq_tt')
def predict_percent_fit(quals_list):
    y_predict_proba = np.array([])
    if len(quals_list):
        X_test = bq_tt.transform(bq_cv.transform(quals_list)).toarray()
        y_predict_proba = basic_quals_clf.predict_proba(X_test)
    
    return y_predict_proba

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\basic_quals_df.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\bq_cv_vocab.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\bq_tt.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\basic_quals_clf.pickle



---
# Look for a fit greater than 80%

In [321]:

# Loop through all the unset %fit values, set them if you can, break for help if you can't
match_series = (hunting_df.percent_fit >= 0.0)
for row_index, row_series in hunting_df[~match_series].iterrows():
    quals_list, job_fitness = print_fit_job(row_index, row_series, basic_quals_dict)
    if job_fitness > 0.8:
        if all(qual_str in basic_quals_dict for qual_str in quals_list):
            hunting_df.loc[row_index, 'percent_fit'] = eval(' + '.join(map(qual_sum, quals_list))) / len(quals_list)
            s.store_objects(hunting_df=hunting_df)
        else:
            break
    else:
        if len(quals_list):
            hunting_df.loc[row_index, 'percent_fit'] = eval(' + '.join(map(qual_sum, quals_list))) / len(quals_list)
            s.store_objects(hunting_df=hunting_df)
print('{}/{} = {}% completed'.format(hunting_df[match_series].shape[0], hunting_df.shape[0],
                                     int(100 * hunting_df[match_series].shape[0] / hunting_df.shape[0])))

4244/4244 = 100% completed


In [318]:

# Manually label the unscored qual
qualification_str = quals_list[3]
print(qualification_str)
basic_quals_dict[qualification_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)

Knowledge of CONUS military installations
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\basic_quals_dict.pickle



---
# Rescore the quals dataframe

In [ ]:

from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import time

# Get the training data and models
X = tfidf_matrix.toarray()
y = basic_quals_df.is_fit.to_numpy()
estimators_list = [AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0, n_estimators=50, random_state=None),
                   BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False, max_features=1.0, max_samples=1.0,
                                     n_estimators=10, n_jobs=None, oob_score=False, random_state=None, verbose=0, warm_start=False),
                   ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None,
                                        max_features='auto', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0,
                                        min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None, oob_score=False, random_state=None, verbose=0, warm_start=False),
                   GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None, learning_rate=0.1, loss='deviance', max_depth=3,
                                              max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
                                              min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100,
                                              n_iter_no_change=None, presort='deprecated', random_state=None, subsample=1.0, tol=0.0001,
                                              validation_fraction=0.1, verbose=0, warm_start=False),
                   RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features='auto',
                                          max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None,
                                          min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
                                          oob_score=False, random_state=None, verbose=0, warm_start=False),
                   LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=None, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False),
                   SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3,
                       gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001, verbose=False)]

# Fit the data and add the duration and fitted models to lists
fit_estimators_list = []
training_durations_list = []
for clf in estimators_list:
    start_time = time.time()
    fit_estimators_list.append(clf.fit(X, y))
    stop_time = time.time()
    training_durations_list.append(stop_time - start_time)
s.store_objects(estimators_list=fit_estimators_list, training_durations_list=training_durations_list)

In [ ]:

estimators_list = s.load_object('estimators_list')
inference_durations_list = []
for clf in estimators_list:
    clf_name = str(type(clf)).split('.')[-1].split("'")[0]
    basic_quals_df[clf_name] = np.nan
    start_time = time.time()
    for row_index, row_series in basic_quals_df.iterrows():
        qualification_str = row_series.qualification_str
        X_test = bq_tt.transform(bq_cv.transform([qualification_str])).toarray()
        y_predict_proba = clf.predict_proba(X_test)[0][1]
        basic_quals_df.loc[row_index, clf_name] = y_predict_proba
    stop_time = time.time()
    inference_durations_list.append(stop_time - start_time)
s.store_objects(basic_quals_df=basic_quals_df, inference_durations_list=inference_durations_list)

In [ ]:

%run ../../load_magic/storage.py
from sklearn.ensemble import StackingClassifier
import numpy as np
import time
from sklearn.feature_extraction.text import CountVectorizer

s = Storage()

estimators_list = s.load_object('estimators_list')
clf = StackingClassifier(estimators=[(str(type(e)).split('.')[-1].split("'")[0], e) for e in estimators_list],
                         final_estimator=None, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)
clf_name = str(type(clf)).split('.')[-1].split("'")[0]
basic_quals_df = s.load_object('basic_quals_df')
basic_quals_df[clf_name] = np.nan
fit_estimators_list = estimators_list.copy()
bq_cv_vocab = s.load_object('bq_cv_vocab')
bq_cv = CountVectorizer(vocabulary=bq_cv_vocab)
bq_cv._validate_vocabulary()
bq_tt = s.load_object('bq_tt')
X = bq_tt.transform(bq_cv.transform(basic_quals_df.qualification_str.tolist())).toarray()
y = basic_quals_df.is_fit.to_numpy()
start_time = time.time()
fit_estimators_list.append(clf.fit(X, y))
stop_time = time.time()
training_durations_list = s.load_object('training_durations_list')
training_durations_list.append(stop_time - start_time)
s.store_objects(fit_estimators_list=fit_estimators_list, training_durations_list=training_durations_list)

# Re-score the quals dataframe
inference_durations_list = s.load_object('inference_durations_list')
start_time = time.time()
for row_index, row_series in basic_quals_df.iterrows():
    qualification_str = row_series.qualification_str
    X_test = bq_tt.transform(bq_cv.transform([qualification_str])).toarray()
    y_predict_proba = clf.predict_proba(X_test)[0][1]
    basic_quals_df.loc[row_index, clf_name] = y_predict_proba
stop_time = time.time()
inference_durations_list.append(stop_time - start_time)
s.store_objects(basic_quals_df=basic_quals_df, inference_durations_list=inference_durations_list)

In [ ]:

from sklearn.ensemble import VotingClassifier

clf = VotingClassifier(estimators=[(str(type(e)).split('.')[-1].split("'")[0], e) for e in estimators_list],
                       voting='soft', weights=None, n_jobs=None, flatten_transform=True)
clf_name = str(type(clf)).split('.')[-1].split("'")[0]
basic_quals_df[clf_name] = np.nan
fit_estimators_list = s.load_object('fit_estimators_list')
start_time = time.time()
fit_estimators_list.append(clf.fit(X, y))
stop_time = time.time()
training_durations_list = s.load_object('training_durations_list')
training_durations_list.append(stop_time - start_time)
s.store_objects(fit_estimators_list=fit_estimators_list, training_durations_list=training_durations_list)

# Re-score the quals dataframe
bq_cv_vocab = s.load_object('bq_cv_vocab')
bq_cv = CountVectorizer(vocabulary=bq_cv_vocab)
bq_cv._validate_vocabulary()
bq_tt = s.load_object('bq_tt')
inference_durations_list = s.load_object('inference_durations_list')
start_time = time.time()
for row_index, row_series in basic_quals_df.iterrows():
    qualification_str = row_series.qualification_str
    X_test = bq_tt.transform(bq_cv.transform([qualification_str])).toarray()
    y_predict_proba = clf.predict_proba(X_test)[0][1]
    basic_quals_df.loc[row_index, clf_name] = y_predict_proba
stop_time = time.time()
inference_durations_list.append(stop_time - start_time)
s.store_objects(basic_quals_df=basic_quals_df, inference_durations_list=inference_durations_list)

In [ ]:

print(basic_quals_df.columns.tolist())
basic_quals_df.sample(5).T

In [ ]:

import inspect

metrics_list = ['accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision_score',
                'balanced_accuracy_score', 'cohen_kappa_score', 'completeness_score', 'explained_variance_score',
                'f1_score', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard_score', 'mutual_info_score',
                'normalized_mutual_info_score', 'precision_score', 'r2_score', 'recall_score', 'roc_auc_score', 'v_measure_score']
description_dict = {name: fn.__doc__.strip().split('\n')[0] for name, fn in inspect.getmembers(sys.modules[__name__],
                                                                                               inspect.isfunction) if name in metrics_list}
for name, cls in inspect.getmembers(sys.modules[__name__], inspect.isclass):
    if name in entropy_df.index:
        description_dict[name] = cls.__doc__.strip().split('\n')[0]
s.store_objects(metrics_list=metrics_list, description_dict=description_dict)
exec('from sklearn.metrics import {}'.format(', '.join(metrics_list)))

In [ ]:

from scipy.stats import entropy

fit_estimators_list = s.load_object('fit_estimators_list')
clf_name_list = [str(type(clf)).split('.')[-1].split("'")[0] for clf in fit_estimators_list]
basic_quals_df = s.load_object('basic_quals_df')
y_true = basic_quals_df.is_fit.tolist()
fit_match_series = (basic_quals_df.is_fit == 1)
yes_list = basic_quals_df[fit_match_series].is_fit.tolist()
no_list = basic_quals_df[~fit_match_series].is_fit.tolist()
columns_list = ['clf_name', 'training_duration', 'inference_duration', 'boundary_diff', 'clf_yes_entropy', 'relative_yes_entropy'] + metrics_list
rows_list = []
training_durations_list = s.load_object('training_durations_list')
inference_durations_list = s.load_object('inference_durations_list')
for column_name, training_duration, inference_duration in zip(clf_name_list, training_durations_list, inference_durations_list):
    yes_series = basic_quals_df[fit_match_series][column_name]
    upper_bound = yes_series.min()
    no_series = basic_quals_df[~fit_match_series][column_name]
    lower_bound = no_series.max()
    y_pred = []
    for p in basic_quals_df[column_name]:
        if p > 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    row_dict = {}
    row_dict['clf_name'] = column_name
    row_dict['training_duration'] = training_duration
    row_dict['inference_duration'] = inference_duration
    row_dict['boundary_diff'] = upper_bound-lower_bound
    row_dict['clf_yes_entropy'] = entropy(pk=yes_series.tolist())
    row_dict['relative_yes_entropy'] = entropy(pk=yes_list, qk=yes_series.tolist())
    for metric_str in metrics_list:
        try:
            row_dict[metric_str] = eval('{}(y_true, basic_quals_df[column_name].tolist())'.format(metric_str))
        except Exception as e1:
            try:
                row_dict[metric_str] = eval('{}(y_true, y_pred)'.format(metric_str))
            except Exception as e2:
                row_dict[metric_str] = np.nan
    rows_list.append(row_dict)
entropy_df = pd.DataFrame(rows_list, columns=columns_list).dropna(axis='columns', how='all')
entropy_df.set_index('clf_name', drop=True, inplace=True)
s.store_objects(entropy_df=entropy_df)

In [ ]:

columns_list = ['training_duration', 'inference_duration', 'balanced_accuracy_score', 'r2_score']
entropy_df[columns_list].sort_values('balanced_accuracy_score', ascending=False)

In [ ]:

entropy_df.columns.tolist()

In [ ]:

fit_estimators_list = s.load_object('fit_estimators_list')
fit_estimators_dict = {str(type(clf)).split('.')[-1].split("'")[0]: clf for clf in fit_estimators_list}
s.store_objects(fit_estimators_dict=fit_estimators_dict)

In [ ]:

%run ../../load_magic/storage.py

s = Storage()
entropy_df = s.load_object('entropy_df')
metrics_list = s.load_object('metrics_list')
custom_metrics_list = ['boundary_diff', 'clf_yes_entropy', 'relative_yes_entropy']
columns_list = metrics_list + custom_metrics_list
columns_list = [cn for cn, s in sorted([(cn, entropy_df[cn].std()) for cn in columns_list], key=lambda x: x[1], reverse=True)][:3]
for metric in columns_list:
    print('{}: {}'.format(metric, description_dict[metric]))
AxesSubplot_obj = entropy_df[columns_list].sort_values('r2_score', ascending=True).plot.line(rot=45, figsize=(18, 8))

In [ ]:

import matplotlib.pyplot as plt

description_dict = s.load_object('description_dict')
columns_list = ['training_duration', 'inference_duration', 'balanced_accuracy_score', 'r2_score']
for metric in columns_list:
    if metric in description_dict:
        print('{}: {}'.format(metric, description_dict[metric]))
entropy_df = s.load_object('entropy_df')
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_yscale('log')
AxesSubplot_obj = entropy_df[columns_list].sort_values('training_duration', ascending=True).plot.line(rot=45, ax=ax)

In [ ]:

metrics_list = s.load_object('metrics_list')
columns_list = [cn for cn in metrics_list if 'accur' in cn.lower()]
for metric in columns_list:
    if metric in description_dict:
        print('{}: {}'.format(metric, description_dict[metric]))
AxesSubplot_obj = entropy_df[columns_list].sort_values('accuracy_score', ascending=True).plot.line(rot=45, figsize=(18, 8))

In [ ]:

entropy_df[custom_metrics_list].sort_values('boundary_diff', ascending=False)

In [ ]:

for metric in custom_metrics_list:
    if metric in description_dict:
        print('{}: {}'.format(metric, description_dict[metric]))
AxesSubplot_obj = entropy_df[custom_metrics_list].sort_values('boundary_diff', ascending=True).plot.line(rot=45, figsize=(18, 8))


---
# Add Next ORR

In [17]:

file_name = 'BAH1002 - Open Requisitions Report (ORR) 2020-03-06 09_04 EST.csv'
print(file_name)
hunting_df = add_new_orr(file_name, hunting_df)

BAH1002 - Open Requisitions Report (ORR) 2020-03-06 09_04 EST.csv


In [18]:

hunting_df.tail(5).T

,4239,4240,4241,4242,4243
Hiring Manager,Douglas Persons | Doug Persons (543133),Douglas Persons | Doug Persons (543133),Michael Peters (608906),John Harrison (576903),Paul Burton (533981)
Management Level,Associate,Associate,Senior Consultant,Senior Consultant,Senior Consultant
IMT,SERVICES MI IMT,SERVICES MI IMT,PS TALENT ACQUISITION IMT,JCC IMT,NA DIGITAL SOLUTIONS IMT
Job Requisition,R0079159 UI/UX Engineer (Open),"R0079160 UI/UX Engineer, Senior (Open)",R0079724 UNSTOPPABLE - ERP Profile (Open),R0080193 Wargaming Specialist (Open),"R0079769 Web Developer, Mid (Open)"
Job Requisition Type,Sold & Unfunded,Sold & Unfunded,Capability Building,Sold and Funded,Sold and Funded
Cluster,Tampa Cluster,Wash Metro Cluster,Wash Metro Cluster,Colorado Springs Cluster,Wash Metro Cluster
Time Type,Full time,Full time,Full time,Full time,Full time
Job Posting Title,UI/UX Engineer,"UI/UX Engineer, Senior",UNSTOPPABLE - ERP Profile,Wargaming Specialist,"Web Developer, Mid"
Safi Recommendation,NaN,NaN,NaN,NaN,NaN
Recruiting Start Date,2020-02-20,2020-02-20,2020-02-27,2020-03-04,2020-02-27


In [22]:

s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle



---
# Flag setting

In [ ]:

# Set remote delivery for this list of jobs
req_id_list = ['R0073564', 'R0073583', 'R0073584', 'R0073585', 'R0073586']
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[match_series, 'is_remote_delivery'] == True
hunting_df.loc[match_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

In [17]:

# Set remote delivery for this one job
req_id = 'R007921'
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'is_remote_delivery'] == False
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [ ]:

# Set university recruiting for this job
req_id = 'R0066388'
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'is_for_university_recruiting'] = 1
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Manually note you can't do this job from home
req_id = 'R0064764'
match_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[match_series, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Manually note you can't do this job from home
hunting_df.loc[83, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

In [ ]:

match_series = (hunting_df.index == 2616)
hunting_df.loc[match_series, 'is_for_university_recruiting'] = 1
hunting_df.loc[match_series, 'percent_fit'] = 0.0
s.store_objects(hunting_df=hunting_df)


---
# Data Exploration

In [ ]:

req_id = 'R0063832'
print_job_description(req_id)

In [ ]:

def f(x):
    if re.search(r'\bIAT\b', str(x)):
        results = True
    else:
        results = False
    
    return results

match_series = basic_quals_df.qualification_str.map(f)
for qual in basic_quals_df[match_series].qualification_str.tolist():
    print('•\t{} = {}'.format(qual, basic_quals_dict[qual]))

In [ ]:

basic_quals_dict['Ability to operate independently and manage staff'] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)

In [ ]:

req_id = 'R0073507'
match_series = (hunting_df['Job Requisition ID'] == req_id)
print(hunting_df[match_series]['percent_fit'].tolist())
for row_index, row_series in hunting_df[match_series].iterrows():
    quals_list, job_fitness = print_fit_job(row_index, row_series, basic_quals_dict)

In [ ]:

hunting_df.loc[504, 'percent_fit'] = (1+1+1+1+0+1+1)/7
s.store_objects(hunting_df=hunting_df)

In [ ]:

req_id_list = ['R0073564', 'R0073583', 'R0073584', 'R0073585', 'R0073586']
match_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df[match_series].T

In [ ]:

match_series = (hunting_df.index == 437)
print(hunting_df[match_series]['Job Description'].tolist()[0])

In [ ]:

match_series = (hunting_df.percent_fit >= 0.0)
print(hunting_df[~match_series].sample(1)['Job Description'].tolist()[0])

In [ ]:

#print(['hunting_df.{}'.format(fn) for fn in dir(hunting_df) if 'dup' in fn.lower()])
match_series = hunting_df.duplicated(subset='Job Requisition ID', keep=False)
print(hunting_df[match_series].shape)

In [ ]:

columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                'Job Posting Title', 'Recruiting Start Date', 'Account Group', 'Job Requisition ID', 'Job Type',
                'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                'Resource Manager', 'Primary Location', 'Job Description', 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                'Required Clearance', 'Primary Recruiter']
hunting_df = hunting_df.drop_duplicates(subset=columns_list, ignore_index=True)
s.store_objects(hunting_df=hunting_df)

In [ ]:

idx_list = hunting_df[match_series].index.tolist()
first = idx_list[0]
second = idx_list[1]
columns_list = []
for column_name in hunting_df.columns:
    if hunting_df.loc[first, column_name] == hunting_df.loc[second, column_name]:
        columns_list.append(column_name)
columns_list

In [ ]:

match_series = hunting_df.percent_fit.isnull()
print(hunting_df[match_series].shape)
req_id = hunting_df.loc[481, 'Job Requisition ID']
print_job_description(req_id)

In [ ]:

print(['{}'.format(fn) for fn in hunting_df.columns if 'req' in fn.lower()])

In [ ]:

key_regex = re.compile(r'([^0-9A-Za-z\+ \/)(:,]+)-')
for old_key in basic_quals_dict.keys():
    match_obj = key_regex.search(old_key)
    if match_obj:
        print('"{}": {}'.format(match_obj.group(1), old_key))
        #new_key = re.sub('^[?â-]+', '', old_key)
        #print(new_key)
        #basic_quals_dict[new_key] = basic_quals_dict.pop(old_key)
        break

In [ ]:

key_regex = re.compile(r'\s+$')
old_key_list = basic_quals_dict.copy().keys()
for old_key in old_key_list:
    match_obj = key_regex.search(old_key)
    if match_obj:
        #print('"{}": {}'.format(match_obj.group(1), old_key))
        new_key = re.sub(r'\s+$', '', old_key)
        #print(new_key)
        basic_quals_dict[new_key] = basic_quals_dict.pop(old_key)
        #print(old_key)
        #break


---
# Manually score unscored jobs

In [ ]:

hunting_df.loc[436, 'percent_fit'] = (0+1+1+1+1+1)/6
s.store_objects(hunting_df=hunting_df)


---
# Study of the Safi recommendations

In [ ]:

import random

match_series = (hunting_df['Safi Recommendation'] == 1)
[c[10:100].strip() for c in random.choices(population=hunting_df[match_series]['Job Description'].unique(), k=10)]

In [ ]:

s.store_objects(hunting_df=hunting_df)
match_series = (hunting_df['Safi Recommendation'] == 1)
hunting_df[match_series]['Primary Location State/Province'].unique().tolist()

In [ ]:

hunting_df[match_series]['Job Requisition'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Cluster'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Job Family'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Account Group'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Resource Manager'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Job Requisition Type'].unique()[:10].tolist()

In [ ]:

hunting_df[match_series]['Job Posting'].unique()[:10].tolist()

In [ ]:

item_list = population=hunting_df[match_series]['Hiring Manager'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['IMT'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Required Clearance'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Job Profile'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Management Level'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Clearance Agency'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Group'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Job Profile'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

[cn for cn in columns_list if 'loca' in cn.lower()]

In [ ]:

item_list = population=hunting_df[match_series]['Primary Location'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])

In [ ]:

item_list = population=hunting_df[match_series]['Primary Recruiter'].unique().tolist()
if len(item_list) > 10:
    print(random.choices(item_list, k=10))
else:
    print(item_list[:10])


---
# Initial dataframe creation (don't run again)

In [ ]:

hunting_dir = r'D:\Documents\Administrivia\Job Hunting'
columns_list = []
for root, dirs, files in os.walk(hunting_dir):
    #path = root.split(os.sep)
    #print((len(path)-1) * '---', os.path.basename(root))
    for file in files:
        #print(len(path) * '---', file)
        if file.endswith('.csv'):
            print(file)
            file_name = os.path.join(hunting_dir, file)
            if os.path.isfile(file_name):
                df = pd.read_csv(file_name, encoding='iso8859-1')
                columns_list = list(set(columns_list) | set(df.columns.tolist()))

In [ ]:

hunting_df = pd.DataFrame([], columns=columns_list)

for root, dirs, files in os.walk(hunting_dir):
    for file in files:
        if file.endswith('.csv'):
            file_name = os.path.join(hunting_dir, file)
            if os.path.isfile(file_name):
                df = pd.read_csv(file_name, encoding='iso8859-1')
                hunting_df = pd.concat([hunting_df, df])


---

In [ ]:

command_str = '{sys.executable} -m pip install pyOutlook'.format(sys=sys)
print(command_str)
!{command_str}